### Cargo los paquetes nesesarias

In [1]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
import math
####################TABLES########################################################
from tabulate import tabulate
from IPython.display import display, Math, Latex
from IPython.display import HTML
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

# Secciones Eficaces y Branching Ratios

## Secciones Eficaces

In [2]:
#Format:
#Name={'value': , 'stat':  , 'sys': , 'Theo': , 'lum': , 'source': };
Sig_WW_13TeV={'value': 142.0 , 'stat': 5.0  , 'sys': 13.0  , 'Theo': 'NA'  , 'lum': 3.2  , 'source': '<a href="https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/CombinedSummaryPlots/SM/ATLAS_c_SMSummary_TotalXsect_rotated/ATLAS_c_SMSummary_TotalXsect_rotated.pdf">reference</a>'};
Sig_tW_13TeV={'value': 94.0  , 'stat': 10.0  , 'sys': 28.0  , 'Theo': 'NA'  , 'lum': 3.2  , 'source': '<a href="https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/CombinedSummaryPlots/SM/ATLAS_c_SMSummary_TotalXsect_rotated/ATLAS_c_SMSummary_TotalXsect_rotated.pdf">reference</a>'};
Sig_DY_13TeV={'value': 1170.0, 'stat': 60.0 , 'sys': 'NA'  , 'Theo': 'NA'  , 'lum': 'NA' , 'source': '<a href="https://arxiv.org/pdf/1407.3643.pdf">Pag 11</a>'};
Sig_tt_13TeV={'value': 835.0 , 'stat': 25.0 , 'sys': 118.0 , 'Theo': 'NA'  , 'lum': 23.0 , 'source': '<a href="https://twiki.cern.ch/twiki/pub/CMSPublic/PhysicsResultsTOPSummaryFigures/Top_Summary_13TeV.pdf">reference</a>'};
Sig_WZ_13TeV={'value': 50.6  , 'stat': 26.0 , 'sys': 2.5 , 'Theo': 'NA'  , 'lum': 23.0 , 'source': '<a href="https://twiki.cern.ch/twiki/pub/CMSPublic/PhysicsResultsTOPSummaryFigures/Top_Summary_13TeV.pdf">reference</a>'};
Cross_Sections={'Sig_WW_13TeV':Sig_WW_13TeV,'Sig_tW_13TeV':Sig_tW_13TeV,'Sig_DY_13TeV':Sig_DY_13TeV,'Sig_tt_13TeV':Sig_tt_13TeV};

## Branching Ratios

In [3]:
Br_t_Wb=  {'value': 95.7  , 'sys': 7.0  , 'lum': 'NA' , 'Theo': 'NA'  , 'lum': 'NA' , 'source':'<a href="http://iopscience.iop.org/1674-1137/40/10/100001">pag 1.981</a>'}
Br_W_munu={'value': 10.63 , 'sys': 0.15 , 'lum': 'NA' , 'Theo': 'NA'  , 'lum': 'NA' , 'source':'<a href="http://iopscience.iop.org/1674-1137/40/10/100001">pag 617</a>'}
Br_Z_mumu={'value': 3.366 , 'sys': 0.007 , 'lum': 'NA' , 'Theo': 'NA'  , 'lum': 'NA' , 'source':'<a href="http://iopscience.iop.org/1674-1137/40/10/100001">pag 30</a>'}
Br_W_jj={'value': 67.41 , 'sys': 0.27 , 'lum': 'NA' , 'Theo': 'NA'  , 'lum': 'NA' , 'source':'<a href="http://iopscience.iop.org/1674-1137/40/10/100001">pag 29</a>'}

Br={'Br_t_Wb':Br_t_Wb,'Br_W_munu':Br_W_munu,'Br_Z_mumu':Br_Z_mumu};

## Secciones Eficaces Finales

Se Construllen las secciones eficaces para cada uno de los procesos

In [ ]:
ALL_Cross_Sections={};
ALL_Cross_Sections['BackGround-DY_2j-Run0']=Cross_Sections['Sig_DY_13TeV'];
ALL_Cross_Sections['BackGround-tW-Run0']=Cross_Sections['Sig_tW_13TeV'];
ALL_Cross_Sections['BackGround-WW-Run0']=Cross_Sections['Sig_WW_13TeV'];
ALL_Cross_Sections['BackGround-mumuW']=Cross_Sections['Sig_WW_13TeV'];
ALL_Cross_Sections['Signal_2j_mu-nu-Run0']=Cross_Sections['Sig_tt_13TeV'];

## Funciones para generar las tablas

In [4]:
def TABLA( Z, formato= '.3e', columna1='Cuts'):
    """Formato es el formato de los numeros, ejem: .2f , .4e"""
    tabla=[];
    head=Z.keys();
    head.sort();
    tmp=Z[Z.keys()[0]].keys();
    tmp.sort();
    for i in tmp:
        entries=[];
        entries.append(i);
        for j in head:
            entries.append(Z[j][i]);
        tabla.append(entries);
    head.insert(0, columna1);
    tabla.insert(0,head);
    
    return HTML(tabulate(tabla, headers= "firstrow", floatfmt= formato, tablefmt='html',showindex="always"))

In [5]:
#Para ver el el diccionario sin una etrada
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

# Análisis de datos utilizando PyRoot


### Cargo las Librerías nesesarias

In [6]:
ROOT.gROOT.Reset()
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#Solo funciona con la versión de Delphes compilada con root 5 (y obiamente root 5)
ROOT.gSystem.AddDynamicPath("~/HEPTools/Delphes/delphes5/")
ROOT.gSystem.Load("libDelphes");

### Construcción de los TChain

In [7]:
#Lista de carpetas le analisis
#Path1= "/scratch/cms/Datos/" #Inportante dejar aquí el último /, PARA CUANDO SE TRABAJA EN GFIF
Path1= "/scratch/cms/cms_gfif/Datos/" #Inportante dejar aquí el último /, PARA CUANDO SE REMOTAMENTE
Path2= "/Events"
Path3= "/tag_1_delphes_events.root"
ListOfFolders = ["BackGround-DY_2j-Run0","BackGround-tW-Run0","BackGround-WW-Run0","Signal_2j_mu-nu-Run0","BackGround-mumuW"]


In [8]:
#Para guardar la info sobre los cortes
signal={};
for Folder in ListOfFolders:
    comand ="ls " + Path1 + Folder + Path2;
    tmp = commands.getstatusoutput( comand )
    
    #Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
    Runs = tmp[1].split("\n");
    
    #Creo el objeto TChain
    MainChain=ROOT.TChain("Delphes")
    
    #Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
    for run in range(len(Runs)):
        imput = Path1 + Folder + Path2 + "/" + Runs[run] + "/tag_1_delphes_events.root"
        MainChain.Add(imput)
        
    #Creo el objeto ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(MainChain)
    numberOfEntries = treeReader.GetEntries()
    
    #Obtengo las ramas que voy a usar
    branchJet = treeReader.UseBranch("Jet")
    branchMuon = treeReader.UseBranch("Muon")
    branchMissingET = treeReader.UseBranch("MissingET")

    # Inicializo los contadores
    cut1=0; cut2=0; cut3=0; cut4=0; cut5=0;
    cuts={};
    
    # Loop sobre los eventos
    for entry in range(0, numberOfEntries):
        #Se carga la entrada espesífica
        treeReader.ReadEntry(entry)
        
        # Primer corte, al menos 2 jet
        if branchJet.GetEntries() > 1:
            cut1 = cut1 +1;
            bcount=0;
            for i in range(0, branchJet.GetEntries() ): #Esta parte es la que produce el error (TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available)
                jet = branchJet.At(i)
                #Cuento cuantos jet hay identificados como b quarks
                if jet.BTag :
                    bcount = bcount + 1;
            
            # Segundo corte, al menos 2 jet b       
            if bcount > 1:
                cut2 = cut2 +1;
            
                # If event contains at least 2 muons
                if branchMuon.GetEntries() > 1:
                    cut3 = cut3 +1;
                    # Tomo los dos muones mas energéticos y comparo sus cargas eléctricas
                    muon1 = branchMuon.At(0)
                    muon2 = branchMuon.At(1)
                
                    #Corte 4: Que los muones mas energéticos tengan cargas diferentes
                    if muon1.Charge*muon2.Charge < 0:
                        cut4 = cut4 +1;

                    
                        #Corte 5: muones mas energéticos tengan masa invariante diferente de la del Z
                        MassMuon= (muon1.P4()+muon2.P4()).M();
                        if not((MassMuon>80) and (MassMuon<100)):
                            cut5 = cut5 +1;
    

    cuts['C0']=numberOfEntries;
    cuts['C1'] = cut1;
    cuts['C2'] = cut2;
    cuts['C3'] = cut3;
    cuts['C4'] = cut4;
    cuts['C5'] = cut5; 
    signal[Folder]=cuts

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available


Donde Co,C1,C2,... Son el número de eventos, sin cortes, con el primer corte, segundo y asi susecivamente.

In [9]:
EffA={};  #Eficiencia Acumulada
for entry in signal.keys():
    EffCut={};  
    for i in signal[entry].keys():
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry]['C0']);
    EffA['EffA_'+entry] = EffCut;

In [10]:
Effr={};  #Eficiencia Relativa
a=signal[entry].keys()
a.sort()
for entry in signal.keys():
    EffCut={};
    j=0;
    a1=a[j];
    for i in a[1:]:
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry][a1]);
        a1=a[j];
        j=j+1;
    Effr['Effr_'+entry] = EffCut;

In [11]:
Z={};  #Sensitividad
a=signal.keys()
a.remove('Signal_2j_mu-nu-Run0')
a.sort()
B=0;
for entry in a:
    Zi={};  
    b=signal[entry].keys()
    b.sort()
    for i in signal[entry].keys():
        Zi[i] = (0.1*signal['Signal_2j_mu-nu-Run0'][i])/(math.sqrt((0.1*signal['Signal_2j_mu-nu-Run0'][i])+(0.1*signal[entry][i])));
        B=(0.1*signal[entry][i])+B;
    Z["Sensitivity_"+entry] = Zi;
    ZZ = (0.1*signal['Signal_2j_mu-nu-Run0'][i])/(math.sqrt((0.1*signal['Signal_2j_mu-nu-Run0'][i])+B));

In [12]:
TABLA(signal)

,Cuts,BackGround-DY_2j-Run0,BackGround-WW-Run0,BackGround-mumuW,BackGround-tW-Run0,Signal_2j_mu-nu-Run0
0,C0,160448,85553,325964,80417,355148
1,C1,28247,74134,238907,76727,341827
2,C2,317,25633,511,27754,126190
3,C3,137,11387,196,12751,55688
4,C4,137,11387,196,12750,55687
5,C5,14,9838,10,10985,48077


In [13]:
TABLA(Effr, '.3f')

,Cuts,Effr_BackGround-DY_2j-Run0,Effr_BackGround-WW-Run0,Effr_BackGround-mumuW,Effr_BackGround-tW-Run0,Effr_Signal_2j_mu-nu-Run0
0,C1,0.176,0.867,0.733,0.954,0.962
1,C2,0.002,0.300,0.002,0.345,0.355
2,C3,0.005,0.154,0.001,0.166,0.163
3,C4,0.432,0.444,0.384,0.459,0.441
4,C5,0.102,0.864,0.051,0.862,0.863


In [14]:
TABLA(EffA, '.3f')

,Cuts,EffA_BackGround-DY_2j-Run0,EffA_BackGround-WW-Run0,EffA_BackGround-mumuW,EffA_BackGround-tW-Run0,EffA_Signal_2j_mu-nu-Run0
0,C0,1.000,1.000,1.000,1.000,1.000
1,C1,0.176,0.867,0.733,0.954,0.962
2,C2,0.002,0.300,0.002,0.345,0.355
3,C3,0.001,0.133,0.001,0.159,0.157
4,C4,0.001,0.133,0.001,0.159,0.157
5,C5,0.000,0.115,0.000,0.137,0.135


In [15]:
TABLA(Z, '.3f')

,Cuts,Sensitivity_BackGround-DY_2j-Run0,Sensitivity_BackGround-WW-Run0,Sensitivity_BackGround-mumuW,Sensitivity_BackGround-tW-Run0
0,C0,156.406,169.175,136.082,170.170
1,C1,177.690,167.602,141.846,167.082
2,C2,112.193,102.413,112.108,101.705
3,C3,74.533,67.996,74.493,67.315
4,C4,74.532,67.995,74.493,67.314
5,C5,69.327,63.175,69.330,62.558


## Cross sections and Branchings Ratios

In [16]:
TABLA(Cross_Sections, '.3f', 'Item')

,Item,Sig_DY_13TeV,Sig_WW_13TeV,Sig_tW_13TeV,Sig_tt_13TeV
0,Theo,NA,NA,NA,NA
1,lum,NA,3.2,3.2,23.0
2,source,Pag 11,reference,reference,reference
3,stat,60.0,5.0,10.0,25.0
4,sys,NA,13.0,28.0,118.0
5,value,1170.0,142.0,94.0,835.0


In [17]:
TABLA(Br, '.3f','Item')

,Item,Br_W_munu,Br_t_Wb
0,Theo,NA,NA
1,lum,NA,NA
2,source,pag 617,pag 1.981
3,sys,0.15,7.0
4,value,10.63,95.7


### Función para normalizar a una luminocidad

In [18]:
def norm(lum, Nevent , CS):
    """Función para normalizar el numero de eventos, [lum]=fb^{-1}, [sigma]=pb"""
    lumpb=lum*1000.0;
    Cte=(lumpb*CS)/Nevent
    return Cte;  

In [51]:
def Lum(CS,NEven):
    """Calcula la Luminosidad [lum]=fb^{-1} dado un numero de eventos y la sección eficaz de producción [Sigma]=pb"""
    lumifb=[]
    for i in NEven:
        lumi=round((i/CS)*0.001,2);
        lumifb.append(lumi);
    return lumifb;

In [52]:
table=[['Proceso','$\sigma[pb] $','NEvent [C0,C1,C2,C3,C4,C5]','$\mathcal{L} [fb^{-1}]$','Reference'],
       ['Sig_DY_13TeV',Cross_Sections['Sig_DY_13TeV']['value'],[160448,28247,317,137,137,14],Lum(Cross_Sections['Sig_DY_13TeV']['value'],[160448,28247,317,137,137,14]),Cross_Sections['Sig_DY_13TeV']['source']],
       ['Sig_tW_13TeV',Cross_Sections['Sig_tW_13TeV']['value'],[80417,76727,27754,12751,12750,10985],Lum(Cross_Sections['Sig_tW_13TeV']['value'],[80417,76727,27754,12751,12750,10985]),Cross_Sections['Sig_tW_13TeV']['source']],
       ['Sig_WW_13TeV',Cross_Sections['Sig_WW_13TeV']['value'],[85553,74134,25633,11387,11387,9838],Lum(Cross_Sections['Sig_WW_13TeV']['value'],[85553,74134,25633,11387,11387,9838]),Cross_Sections['Sig_WW_13TeV']['source']]
      ]

In [53]:
HTML(tabulate(table, tablefmt="html",floatfmt='.3e',stralign='center'))

Proceso,$\sigma[pb] $,"NEvent [C0,C1,C2,C3,C4,C5]",$\mathcal{L} [fb^{-1}]$,Reference
Sig_DY_13TeV,1170.0,"[160448, 28247, 317, 137, 137, 14]","[0.14, 0.02, 0.0, 0.0, 0.0, 0.0]",Pag 11
Sig_tW_13TeV,94.0,"[156.406, 177.69, 112.193, 74.533, 74.532, 69.327]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",reference
Sig_WW_13TeV,142.0,"[85553, 74134, 25633, 11387, 11387, 9838]","[0.6, 0.52, 0.18, 0.08, 0.08, 0.07]",reference


In [50]:
ALL_Cross_Sections

{'BackGround-DY_2j-Run0': {'Theo': 'NA',
  'lum': 'NA',
  'source': '<a href="https://arxiv.org/pdf/1407.3643.pdf">Pag 11</a>',
  'stat': 60.0,
  'sys': 'NA',
  'value': 1170.0}}

In [ ]:
table=[['Proceso','$\sigma[pb] $','NEvent [C0,C1,C2,C3,C4,C5]','$\mathcal{L} [fb^{-1}]$','Reference']];
for proces in Cross_Sections.keys():
    Line=[proces,Cross_Sections[proces]['value'],[160448,28247,317,137,137,14],Lum(Cross_Sections['Sig_DY_13TeV']['value'],[160448,28247,317,137,137,14]),Cross_Sections['Sig_DY_13TeV']['source']]];
    

In [45]:
A=signal['BackGround-DY_2j-Run0'].values()

In [46]:
A.reverse()
A

[137, 14, 160448L, 28247, 317, 137]

In [41]:
signal

{'BackGround-DY_2j-Run0': {'C0': 160448L,
  'C1': 28247,
  'C2': 317,
  'C3': 137,
  'C4': 137,
  'C5': 14},
 'BackGround-WW-Run0': {'C0': 85553L,
  'C1': 74134,
  'C2': 25633,
  'C3': 11387,
  'C4': 11387,
  'C5': 9838},
 'BackGround-mumuW': {'C0': 325964L,
  'C1': 238907,
  'C2': 511,
  'C3': 196,
  'C4': 196,
  'C5': 10},
 'BackGround-tW-Run0': {'C0': 80417L,
  'C1': 76727,
  'C2': 27754,
  'C3': 12751,
  'C4': 12750,
  'C5': 10985},
 'Signal_2j_mu-nu-Run0': {'C0': 355148L,
  'C1': 341827,
  'C2': 126190,
  'C3': 55688,
  'C4': 55687,
  'C5': 48077}}